In [1]:
!pip install -q watermark
!pip install matplotlib seaborn

In [2]:
%load_ext watermark
%watermark -p torch,transformers,pandas

torch       : 1.13.0
transformers: 4.27.4
pandas      : 1.3.5



In [3]:
import re
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.auto import tqdm
import seaborn as sns
from transformers import AutoTokenizer, AutoModel

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Config

In [4]:
# Model
MODEL_CKPT = 'bert-base-uncased'

# Hyperparameters
MAX_LEN = 320
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE * 2
EPOCHS = 2
LEARNING_RATE = 1e-05
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print("Device:", DEVICE)

Device: cuda:0


In [5]:
FOR_SUBMISSION = True  # `False` for experimenting

# Load and Prepare Dataset

In [6]:
train_data = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv')
print("Num. samples:", len(train_data))

Num. samples: 159571


In [7]:
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_data['labels'] = train_data[label_columns].apply(lambda x: list(x), axis=1)

train_data.drop(['id'], inplace=True, axis=1)
train_data.drop(label_columns, inplace=True, axis=1)

train_data.head(3)

,comment_text,labels
0,Explanation\nWhy the edits made under my usern...,"[0, 0, 0, 0, 0, 0]"
1,D'aww! He matches this background colour I'm s...,"[0, 0, 0, 0, 0, 0]"
2,"Hey man, I'm really not trying to edit war. It...","[0, 0, 0, 0, 0, 0]"


In [8]:
def clean_text(txt):
    """Perform some basic cleaning of the text."""
    return re.sub("[^A-Za-z0-9.,;:!?]+", ' ', str(txt))

In [9]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len, new_data=False):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.text = dataframe.comment_text
        self.new_data = new_data
        self.max_len = max_len
        
        if not new_data:
            self.targets = self.data.labels

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        text = clean_text(text)

        inputs = self.tokenizer(
            text, 
            truncation=True, 
            padding='max_length' if self.new_data else False,
            max_length=self.max_len, 
            return_tensors="pt"
        )
        inputs = {k: v.squeeze() for k, v in inputs.items()}
        
        if not self.new_data:
            labels = torch.tensor(self.targets[index], dtype=torch.float)
            return inputs, labels

        return inputs

In [10]:
train_size = 1.0 if FOR_SUBMISSION else 0.85

train_df = train_data.sample(frac=train_size, random_state=123)
val_df = train_data.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

print("Orig Dataset: {}".format(train_data.shape))
print("Training Dataset: {}".format(train_df.shape))
print("Validation Dataset: {}".format(val_df.shape))

Orig Dataset: (159571, 2)
Training Dataset: (159571, 2)
Validation Dataset: (0, 2)


In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT, do_lower_case=True)

train_set = MultiLabelDataset(train_df, tokenizer, MAX_LEN)
val_set = MultiLabelDataset(val_df, tokenizer, MAX_LEN)

We've deliberately postponed padding until now, as we want to pad batches on the fly. Here is our custom data collator that will do that:

In [12]:
def dynamic_collate(data):
    """Custom data collator for dynamic padding."""
    inputs = [d for d,l in data]
    labels = torch.stack([l for d,l in data], dim=0)
    inputs = tokenizer.pad(inputs, return_tensors='pt')
    return inputs, labels

In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2, 
                'collate_fn': dynamic_collate}

val_params = {'batch_size': VALID_BATCH_SIZE,
              'shuffle': False,
              'num_workers': 2, 
              'collate_fn': dynamic_collate}

train_loader = DataLoader(train_set, **train_params)
val_loader = None if FOR_SUBMISSION else DataLoader(val_set, **val_params)

# Model

In [14]:
class TransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained(MODEL_CKPT)
        self.classifier = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(768, 6)
        )

    def forward(self, inputs):
        bert_output = self.bert(**inputs)
        hidden_state = bert_output.last_hidden_state
        pooled_out = hidden_state[:, 0]
        logits = self.classifier(pooled_out)
        return logits

model = TransformerModel()
model.to(DEVICE);

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])
print(f"Trainable params: {round(trainable_params/1e6, 1)} M")

Trainable params: 110.1 M


In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_func = nn.BCEWithLogitsLoss()

In [17]:
lr_sched = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.2)
test_df = pd.read_csv('/kaggle/input/tf2chatsunlabelled/chatlog.csv')
test_df.head(80)

,steamid,name,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,[U:1:158517868],Orange-Juice,bad spy,NaN,NaN,NaN,NaN,NaN,NaN
1,[U:1:158517868],Orange-Juice,:(,NaN,NaN,NaN,NaN,NaN,NaN
2,[U:1:236710169],CUNNY PLEASE,loll,NaN,NaN,NaN,NaN,NaN,NaN
3,[U:1:174480808],kris,gg,NaN,NaN,NaN,NaN,NaN,NaN
4,[U:1:236710169],CUNNY PLEASE,insane,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
75,[U:1:378260322],AG,sb,NaN,NaN,NaN,NaN,NaN,NaN
76,[U:1:378260322],AG,sb,NaN,NaN,NaN,NaN,NaN,NaN
77,[U:1:1210179367],小灰灰本人,6,NaN,NaN,NaN,NaN,NaN,NaN
78,[U:1:1210179367],小灰灰本人,.SS,NaN,NaN,NaN,NaN,NaN,NaN


# Training

In [18]:
def accuracy_multi(inp, targ, thresh=0.5, sigmoid=True):
    """An accuracy metric for multi-label problems."""
    if sigmoid: 
        inp = inp.sigmoid()
    return ((inp > thresh) == targ.bool()).float().mean()

In [19]:
def train_one_epoch(train_loader, model, loss_func, optimizer, progress_bar=None):
    """Train model over one epoch."""
    model.train()
    size = len(train_loader.dataset)  # Train set size
    
    for i, (data, targets) in enumerate(train_loader):
        # Put inputs and target on DEVICE
        data = {k: v.to(DEVICE) for k, v in data.items()}
        targets = targets.to(DEVICE)
        
        outputs = model(data)
        loss = loss_func(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if progress_bar is not None:
            progress_bar.update(1)
        
        if i % 1000 == 0:
            loss, step = loss.item(), i * len(targets)
            print(f"Loss: {loss:>4f}  [{step:>6d}/{size:>6d}]")
        elif i == len(train_loader) - 1:
            loss = loss.item()
            print(f"Loss: {loss:>4f}  [{size:>6d}/{size:>6d}]")

In [20]:
def validate_one_epoch(val_loader, model, loss_func):
    """Validate model over one epoch."""
    model.eval()
    num_batches = len(val_loader)
    
    valid_loss, acc_multi = 0, 0

    with torch.no_grad():
        for _, (data, targets) in enumerate(val_loader):
            data = {k: v.to(DEVICE) for k, v in data.items()}
            targets = targets.to(DEVICE)

            outputs = model(data)
            valid_loss += loss_func(outputs, targets).item()
            acc_multi += accuracy_multi(outputs, targets)

    valid_loss /= num_batches  # Avg. loss
    acc_multi /= num_batches   # Avg. acc. multi
    print(f"Avg. valid. loss: {valid_loss:>4f}, Acc. multi: {acc_multi:>4f}\n")
    

In [ ]:
num_train_steps = EPOCHS * len(train_loader)
progress_bar = tqdm(range(num_train_steps))

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1} (lr = {lr_sched.get_last_lr()[0]:.2e})\n-------------------------------")
    train_one_epoch(train_loader, model, loss_func, optimizer, progress_bar)
    if not FOR_SUBMISSION:
        validate_one_epoch(val_loader, model, loss_func)
    lr_sched.step()

  0%|          | 0/9974 [00:00<?, ?it/s]

Epoch 1 (lr = 1.00e-05)
-------------------------------


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Loss: 0.723767  [     0/159571]


# Generate Test Submissions

In [ ]:
test_df = pd.read_csv('/kaggle/input/tf2chatsunlabelled/chatlog.csv')

test_df.head(3)

In [ ]:
test_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': False,
               'num_workers': 2}

test_set = MultiLabelDataset(test_df, tokenizer, MAX_LEN, new_data=True)
test_loader = DataLoader(test_set, **test_params)

In [ ]:
def predict(test_loader, model):
    """Make predictions on test set."""
    model.eval()
    all_preds = []
    
    with torch.inference_mode():
        for data in tqdm(test_loader):
            data = {k: v.to(DEVICE) for k, v in data.items()}

            outputs = model(data)
            probas = torch.sigmoid(outputs)

            all_preds.append(probas)
            
        all_preds = torch.cat(all_preds)
    return all_preds.cpu()

In [ ]:
all_test_pred = predict(test_loader, model)

In [ ]:
submit_df = test_df.copy()

In [ ]:
for i, name in enumerate(label_columns):
    submit_df[name] = all_test_pred[:, i]

submit_df.head()

In [ ]:

submit_df.to_csv('Chatslabelled.csv', index=False)